In [67]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [68]:
!nvidia-smi

Fri May 19 20:14:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [69]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5ir4t8dm
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5ir4t8dm
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done


In [70]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [71]:
!gcc -o MatrixMultSeq.c -lm
!./MatrixMult

/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/9/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x24): undefined reference to `main'
collect2: error: ld returned 1 exit status
/bin/bash: ./MatrixMult: No such file or directory


# Matrix Size 500x500

In [72]:
%%writefile matrixseq.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define SIZE 500

void MatrixMultSeq(float M[SIZE][SIZE], float N[SIZE][SIZE], float P[SIZE][SIZE])
{
    for (int i = 0; i < SIZE; i++) {
        for (int j = 0; j < SIZE; j++) {
            P[i][j] = 0.0;
            for (int k = 0; k < SIZE; k++) {
                P[i][j] += M[i][k] * N[k][j];
            }
        }
    }
}

int main() {
    float M[SIZE][SIZE];
    float N[SIZE][SIZE];
    float P[SIZE][SIZE];

    srand(time(NULL));
    for (int i = 0; i < SIZE; i++) {
        for (int j = 0; j < SIZE; j++) {
            M[i][j] = rand() / (float)RAND_MAX;
            N[i][j] = rand() / (float)RAND_MAX;
        }
    }

    clock_t start, end;
    double cpu_time;

    start = clock();

    MatrixMultSeq(M, N, P);

    end = clock();

    cpu_time = ((double) (end - start)) / CLOCKS_PER_SEC;

    printf("Execution time: %f s\n", cpu_time);

    return 0;
}


Overwriting matrixseq.c


In [73]:
!gcc matrixseq.c -o matrixseq
!./matrixseq

Execution time: 0.569442 s


## CUDA No Tiling Blocksize 16

In [74]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define BLOCK_SIZE 16

__global__ void MatrixMult(float* M, float* N, float* P, int height, int width, int depth)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < height && col < depth) {
        float pvalue = 0;
        for (int k = 0; k < width; k++) {
            pvalue += M[row * width + k] * N[k * depth + col];
        }
        P[row * depth + col] = pvalue;
    }
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)BLOCK_SIZE), ceil(N / (float)BLOCK_SIZE), 1);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMult<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


Kernel execution time: 1.032224 ms



## Cuda C No tiling Blocksize 32

In [75]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define BLOCK_SIZE 32

__global__ void MatrixMult(float* M, float* N, float* P, int height, int width, int depth)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < height && col < depth) {
        float pvalue = 0;
        for (int k = 0; k < width; k++) {
            pvalue += M[row * width + k] * N[k * depth + col];
        }
        P[row * depth + col] = pvalue;
    }
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)BLOCK_SIZE), ceil(N / (float)BLOCK_SIZE), 1);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMult<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


Kernel execution time: 0.833312 ms



## Cuda C No Tiling Block size 64

In [76]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define BLOCK_SIZE 64

__global__ void MatrixMult(float* M, float* N, float* P, int height, int width, int depth)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < height && col < depth) {
        float pvalue = 0;
        for (int k = 0; k < width; k++) {
            pvalue += M[row * width + k] * N[k * depth + col];
        }
        P[row * depth + col] = pvalue;
    }
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)BLOCK_SIZE), ceil(N / (float)BLOCK_SIZE), 1);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMult<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


Kernel execution time: 0.002944 ms



## Cuda C No Tiling Block Size 128

In [77]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define BLOCK_SIZE 128

__global__ void MatrixMult(float* M, float* N, float* P, int height, int width, int depth)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < height && col < depth) {
        float pvalue = 0;
        for (int k = 0; k < width; k++) {
            pvalue += M[row * width + k] * N[k * depth + col];
        }
        P[row * depth + col] = pvalue;
    }
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)BLOCK_SIZE), ceil(N / (float)BLOCK_SIZE), 1);
    dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMult<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


Kernel execution time: 0.002944 ms



## Cuda C Tiling Tile Size 16

In [78]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define TITLE_SIZE 16

// Kernel function with tiling
__global__ void MatrixMultTiled(float* M, float* N, float* P, int width, int height, int depth)
{
    __shared__ float Ms[TITLE_SIZE][TITLE_SIZE];
    __shared__ float Ns[TITLE_SIZE][TITLE_SIZE];

    int bx = blockIdx.x; 
    int by = blockIdx.y;
    int tx = threadIdx.x; 
    int ty = threadIdx.y;

    // Identify the row and column of the P element to work on
    int Row = by * TITLE_SIZE + ty;
    int Col = bx * TITLE_SIZE + tx;

    float Pvalue = 0;

    // Loop over the M and N tiles required to compute the P element
    for (int m = 0; m < (width-1)/TITLE_SIZE+1; ++m) {

        // Collaborative loading of M and N tiles into shared memory
        if (Row < width && m*TITLE_SIZE+tx < height) 
            Ms[ty][tx] = M[Row*width + (m*TITLE_SIZE + tx)];
        else
            Ms[ty][tx] = 0.0;

        if (m*TITLE_SIZE+ty < width && Col < depth)
            Ns[ty][tx] = N[(m*TITLE_SIZE + ty)*depth + Col];
        else
            Ns[ty][tx] = 0.0;

        __syncthreads();

        for (int k = 0; k < TITLE_SIZE; ++k) {
            Pvalue += Ms[ty][k] * Ns[k][tx];
        }
        __syncthreads();
    }
    if (Row < width && Col < depth)
        P[Row*depth + Col] = Pvalue;
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)TITLE_SIZE), ceil(N / (float)TITLE_SIZE), 1);
    dim3 dimBlock(TITLE_SIZE, TITLE_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMultTiled<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


Kernel execution time: 0.757248 ms



## Cuda C Tiling 32

In [79]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define TITLE_SIZE 32

// Kernel function with tiling
__global__ void MatrixMultTiled(float* M, float* N, float* P, int width, int height, int depth)
{
    __shared__ float Ms[TITLE_SIZE][TITLE_SIZE];
    __shared__ float Ns[TITLE_SIZE][TITLE_SIZE];

    int bx = blockIdx.x; 
    int by = blockIdx.y;
    int tx = threadIdx.x; 
    int ty = threadIdx.y;

    // Identify the row and column of the P element to work on
    int Row = by * TITLE_SIZE + ty;
    int Col = bx * TITLE_SIZE + tx;

    float Pvalue = 0;

    // Loop over the M and N tiles required to compute the P element
    for (int m = 0; m < (width-1)/TITLE_SIZE+1; ++m) {

        // Collaborative loading of M and N tiles into shared memory
        if (Row < width && m*TITLE_SIZE+tx < height) 
            Ms[ty][tx] = M[Row*width + (m*TITLE_SIZE + tx)];
        else
            Ms[ty][tx] = 0.0;

        if (m*TITLE_SIZE+ty < width && Col < depth)
            Ns[ty][tx] = N[(m*TITLE_SIZE + ty)*depth + Col];
        else
            Ns[ty][tx] = 0.0;

        __syncthreads();

        for (int k = 0; k < TITLE_SIZE; ++k) {
            Pvalue += Ms[ty][k] * Ns[k][tx];
        }
        __syncthreads();
    }
    if (Row < width && Col < depth)
        P[Row*depth + Col] = Pvalue;
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)TITLE_SIZE), ceil(N / (float)TITLE_SIZE), 1);
    dim3 dimBlock(TITLE_SIZE, TITLE_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMultTiled<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


Kernel execution time: 0.714336 ms



## Cuda C Tiling Size 64

In [80]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define TITLE_SIZE 64

// Kernel function with tiling
__global__ void MatrixMultTiled(float* M, float* N, float* P, int width, int height, int depth)
{
    __shared__ float Ms[TITLE_SIZE][TITLE_SIZE];
    __shared__ float Ns[TITLE_SIZE][TITLE_SIZE];

    int bx = blockIdx.x; 
    int by = blockIdx.y;
    int tx = threadIdx.x; 
    int ty = threadIdx.y;

    // Identify the row and column of the P element to work on
    int Row = by * TITLE_SIZE + ty;
    int Col = bx * TITLE_SIZE + tx;

    float Pvalue = 0;

    // Loop over the M and N tiles required to compute the P element
    for (int m = 0; m < (width-1)/TITLE_SIZE+1; ++m) {

        // Collaborative loading of M and N tiles into shared memory
        if (Row < width && m*TITLE_SIZE+tx < height) 
            Ms[ty][tx] = M[Row*width + (m*TITLE_SIZE + tx)];
        else
            Ms[ty][tx] = 0.0;

        if (m*TITLE_SIZE+ty < width && Col < depth)
            Ns[ty][tx] = N[(m*TITLE_SIZE + ty)*depth + Col];
        else
            Ns[ty][tx] = 0.0;

        __syncthreads();

        for (int k = 0; k < TITLE_SIZE; ++k) {
            Pvalue += Ms[ty][k] * Ns[k][tx];
        }
        __syncthreads();
    }
    if (Row < width && Col < depth)
        P[Row*depth + Col] = Pvalue;
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)TITLE_SIZE), ceil(N / (float)TITLE_SIZE), 1);
    dim3 dimBlock(TITLE_SIZE, TITLE_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMultTiled<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


Kernel execution time: 0.002976 ms



## Cuda C Tile Size 128

In [81]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#define TITLE_SIZE 128

// Kernel function with tiling
__global__ void MatrixMultTiled(float* M, float* N, float* P, int width, int height, int depth)
{
    __shared__ float Ms[TITLE_SIZE][TITLE_SIZE];
    __shared__ float Ns[TITLE_SIZE][TITLE_SIZE];

    int bx = blockIdx.x; 
    int by = blockIdx.y;
    int tx = threadIdx.x; 
    int ty = threadIdx.y;

    // Identify the row and column of the P element to work on
    int Row = by * TITLE_SIZE + ty;
    int Col = bx * TITLE_SIZE + tx;

    float Pvalue = 0;

    // Loop over the M and N tiles required to compute the P element
    for (int m = 0; m < (width-1)/TITLE_SIZE+1; ++m) {

        // Collaborative loading of M and N tiles into shared memory
        if (Row < width && m*TITLE_SIZE+tx < height) 
            Ms[ty][tx] = M[Row*width + (m*TITLE_SIZE + tx)];
        else
            Ms[ty][tx] = 0.0;

        if (m*TITLE_SIZE+ty < width && Col < depth)
            Ns[ty][tx] = N[(m*TITLE_SIZE + ty)*depth + Col];
        else
            Ns[ty][tx] = 0.0;

        __syncthreads();

        for (int k = 0; k < TITLE_SIZE; ++k) {
            Pvalue += Ms[ty][k] * Ns[k][tx];
        }
        __syncthreads();
    }
    if (Row < width && Col < depth)
        P[Row*depth + Col] = Pvalue;
}

int main() {
    int M = 500;
    int N = 500;
    int K = 500;

    //Allocate mem for host
    float *h_M = (float*)malloc(sizeof(float) * M * N);
    float *h_N = (float*)malloc(sizeof(float) * M * K);
    float *h_P = (float*)malloc(sizeof(float) * N * K);

    srand(time(NULL));
    for (int i = 0; i < N * M; i++) {
        h_M[i] = rand() / (float)RAND_MAX;
    }
    for (int i = 0; i < M * K; i++) {
        h_N[i] = rand() / (float)RAND_MAX;
    }

    //Allocate mem for device
    float *d_M, *d_N, *d_P;
    cudaMalloc((void**)&d_N, sizeof(float) * M * K);
    cudaMalloc((void**)&d_M, sizeof(float)* M * N);
    cudaMalloc((void**)&d_P, sizeof(float) * N * K);
    cudaMemcpy(d_M, h_M, sizeof(float) * M * N, cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, sizeof(float) * M * K, cudaMemcpyHostToDevice);
        
    //Set up config for parallelization
    dim3 dimGrid(ceil(K / (float)TITLE_SIZE), ceil(N / (float)TITLE_SIZE), 1);
    dim3 dimBlock(TITLE_SIZE, TITLE_SIZE, 1);

    float elapsed_time;
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //Invoke Kernel
    MatrixMultTiled<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, N, M, K);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&elapsed_time, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    cudaMemcpy(h_P, d_P, sizeof(float)*N*K, cudaMemcpyDeviceToHost);

    printf("Kernel execution time: %f ms\n", elapsed_time);

    //Host mem free
    free(h_M);
    free(h_N);
    free(h_P);

    //Device mem free
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);

    return 0;
}


ptxas error   : Entry function '_Z15MatrixMultTiledPfS_S_iii' uses too much shared data (0x20000 bytes, 0xc000 max)

